In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import pandas as pd
import wandb
from sklearn.metrics import roc_auc_score, make_scorer
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV, KFold
from sklearn.pipeline import Pipeline

from src.architecture.models import make_class_models, make_risk_models
from src.preproc.preproc_utils import FCBFSelector, ToCuPy
from src.utils.utils import setup_hyperparameters

# 0) Setup run hyperparameters and W&B run
config_path = "src/configs/config.yaml"
_, run = setup_hyperparameters(config_path)

wandb: Currently logged in as: juanravm (juanravm-vall-d-hebron-institute-of-oncology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [50]:
from sksurv.util import Surv
from sksurv.metrics import concordance_index_censored

# 1) Loading data
df = pd.read_csv("data/data_train.tsv", sep="\t")

# Filtering variables with too many missing values
na_frac = df.drop(columns=["risk_status", "dfs_status", "dfs_time"]).isna().mean()
keep = na_frac[na_frac <= 0.10].index.tolist()
df = df.loc[:, keep + ["risk_status", "dfs_status", "dfs_time"]]

# Removing samples with any missing value
keep = df.drop(columns=["risk_status", "dfs_status", "dfs_time"]).dropna().index.tolist()
df = df.loc[keep, :]

print(f"Training with {df.shape[0]} samples")

y_class = df["risk_status"].copy()
y_event = df["dfs_status"].copy()
y_time = df["dfs_time"].copy()
X = df.drop(columns=["risk_status", "dfs_status", "dfs_time", "os_status"])
X_class = X.loc[y_class.notna(), :]
y_class = y_class.loc[y_class.notna()]

X_surv = X.loc[(y_event.notna()) & (y_time.notna()), :]
y_time = y_time.loc[(y_event.notna()) & (y_time.notna())]
y_event = y_event.loc[(y_event.notna()) & (y_time.notna())]
y_surv = Surv.from_arrays(event=y_event.astype(bool), time=y_time)

# 4) Selector for best features
selector = FCBFSelector(mode="rank", threshold=0.0, n_bins=2)

# 5) Models and hyperparameters search
class_models = make_class_models()
risk_models = make_risk_models()

cv_class = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
scorer_class = make_scorer(roc_auc_score)
cv_risk = KFold(n_splits=5, shuffle=True, random_state=0)


def cindex_scorer(estimator, X, y_surv):
    pred = estimator.predict(X)
    return concordance_index_censored(y_surv["event"], y_surv["time"], pred)[0]


scorer_risk = cindex_scorer

Training with 125 samples


In [51]:
from sklearn.base import is_classifier, is_regressor


# 6) Class models
class_results = {}
class_table_rows = []
# y_class = y_class.astype(int).to_numpy().ravel()

for name, (estimator, search_space) in class_models.items():
    if name == "xgb":
        pipe = Pipeline(
            [
                ("selector", selector),
                ("tocupy", ToCuPy()),
                ("model", estimator),
            ]
        )
    else:
        pipe = Pipeline(
            [
                ("selector", selector),
                ("model", estimator),
            ]
        )

    search = RandomizedSearchCV(
        pipe,
        param_distributions=search_space,
        n_iter=50,
        cv=cv_class,
        scoring="roc_auc",
        n_jobs=-1,
        random_state=0,
        refit=True,
    )

    search.fit(X_class, y_class)
    class_results[name] = {
        "best_score": search.best_score_,
        "best_params": search.best_params_,
        "best_estimator": search.best_estimator_,
    }
    print(f"{name}: AUC={search.best_score_:.3f}")
    if run:
        run.log(
            {
                f"class/{name}/best_auc": search.best_score_,
                f"class/{name}/best_params": json.dumps(search.best_params_, default=str),
            }
        )
        class_table_rows.append([name, search.best_score_, json.dumps(search.best_params_, default=str)])

/mnt/CCBdata/projects/conda_envs/BC/lib/python3.11/site-packages/sklearn/preprocessing/_discretization.py:296: FutureWarning: The current default behavior, quantile_method='linear', will be changed to quantile_method='averaged_inverted_cdf' in scikit-learn version 1.9 to naturally support sample weight equivalence properties by default. Pass quantile_method='averaged_inverted_cdf' explicitly to silence this warning.
  warnings.warn(
/mnt/CCBdata/projects/conda_envs/BC/lib/python3.11/site-packages/sklearn/preprocessing/_discretization.py:296: FutureWarning: The current default behavior, quantile_method='linear', will be changed to quantile_method='averaged_inverted_cdf' in scikit-learn version 1.9 to naturally support sample weight equivalence properties by default. Pass quantile_method='averaged_inverted_cdf' explicitly to silence this warning.
  warnings.warn(
/mnt/CCBdata/projects/conda_envs/BC/lib/python3.11/site-packages/sklearn/preprocessing/_discretization.py:296: FutureWarning: 

logreg: AUC=0.884


/mnt/CCBdata/projects/conda_envs/BC/lib/python3.11/site-packages/sklearn/preprocessing/_discretization.py:296: FutureWarning: The current default behavior, quantile_method='linear', will be changed to quantile_method='averaged_inverted_cdf' in scikit-learn version 1.9 to naturally support sample weight equivalence properties by default. Pass quantile_method='averaged_inverted_cdf' explicitly to silence this warning.
  warnings.warn(
/mnt/CCBdata/projects/conda_envs/BC/lib/python3.11/site-packages/sklearn/preprocessing/_discretization.py:296: FutureWarning: The current default behavior, quantile_method='linear', will be changed to quantile_method='averaged_inverted_cdf' in scikit-learn version 1.9 to naturally support sample weight equivalence properties by default. Pass quantile_method='averaged_inverted_cdf' explicitly to silence this warning.
  warnings.warn(
/mnt/CCBdata/projects/conda_envs/BC/lib/python3.11/site-packages/sklearn/preprocessing/_discretization.py:296: FutureWarning: 

svc: AUC=0.902


/mnt/CCBdata/projects/conda_envs/BC/lib/python3.11/site-packages/sklearn/preprocessing/_discretization.py:296: FutureWarning: The current default behavior, quantile_method='linear', will be changed to quantile_method='averaged_inverted_cdf' in scikit-learn version 1.9 to naturally support sample weight equivalence properties by default. Pass quantile_method='averaged_inverted_cdf' explicitly to silence this warning.
  warnings.warn(
/mnt/CCBdata/projects/conda_envs/BC/lib/python3.11/site-packages/sklearn/preprocessing/_discretization.py:296: FutureWarning: The current default behavior, quantile_method='linear', will be changed to quantile_method='averaged_inverted_cdf' in scikit-learn version 1.9 to naturally support sample weight equivalence properties by default. Pass quantile_method='averaged_inverted_cdf' explicitly to silence this warning.
  warnings.warn(
/mnt/CCBdata/projects/conda_envs/BC/lib/python3.11/site-packages/sklearn/preprocessing/_discretization.py:296: FutureWarning: 

rf: AUC=0.909


AttributeError: 'XGBModel' object has no attribute 'feature_weights'

In [7]:
class_results

{'logreg': {'best_score': 0.8522435897435898,
  'best_params': {'selector__kbest': 10,
   'model__l1_ratio': 0.55,
   'model__C': 26.826957952797244},
  'best_estimator': Pipeline(steps=[('selector', FCBFSelector(kbest=10, mode='rank')),
                  ('model',
                   LogisticRegression(C=26.826957952797244, l1_ratio=0.55,
                                      max_iter=5000, penalty='elasticnet',
                                      solver='saga'))])},
 'svc': {'best_score': 0.8749999999999998,
  'best_params': {'selector__kbest': 20,
   'model__kernel': 'rbf',
   'model__gamma': 0.001,
   'model__C': 0.01},
  'best_estimator': Pipeline(steps=[('selector', FCBFSelector(kbest=20, mode='rank')),
                  ('model', SVC(C=0.01, gamma=0.001, probability=True))])},
 'rf': {'best_score': 0.8972756410256411,
  'best_params': {'selector__kbest': 40,
   'model__n_estimators': 200,
   'model__min_samples_split': 2,
   'model__max_features': 'sqrt',
   'model__max_depth':

In [4]:
risk_models = make_risk_models()
selector = FCBFSelector(mode="rank", threshold=0.0, n_bins=2)

In [6]:
from sklearn.model_selection import cross_val_score
import numpy as np
import xgboost as xgb

# 7) Risk models
risk_results = {}
risk_table_rows = []

for name, (estimator, search_space) in risk_models.items():

    # Pipeline
    pipe = Pipeline([
        ("selector", selector),
        ("model", estimator),
    ])

    # Caso 2: con hiperparámetros -> RandomizedSearchCV
    # Caso especial: XGB survival:cox no acepta y_surv (structured)
    search = RandomizedSearchCV(
        pipe,
        param_distributions=search_space,
        n_iter=50,
        cv=cv_risk,
        scoring=scorer_risk,
        n_jobs=-1,
        random_state=0,
        refit=True,
        error_score="raise",
    )
    search.fit(X_surv, y_surv)

    best_score = float(search.best_score_)
    best_params = search.best_params_
    best_estimator = search.best_estimator_

    # Guardar resultados
    risk_results[name] = {
        "best_score": best_score,
        "best_params": best_params,
        "best_estimator": best_estimator,
    }
    print(f"{name}: C-index={best_score:.3f}")

    if run:
        run.log({
            f"risk/{name}/best_cindex": best_score,
            f"risk/{name}/best_params": json.dumps(best_params, default=str),
        })
        risk_table_rows.append([name, best_score, json.dumps(best_params, default=str)])

/mnt/CCBdata/projects/conda_envs/BC/lib/python3.11/site-packages/sklearn/model_selection/_search.py:317: UserWarning: The total space of parameters 30 is smaller than n_iter=50. Running 30 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/mnt/CCBdata/projects/conda_envs/BC/lib/python3.11/site-packages/sksurv/linear_model/coxph.py:197: RuntimeWarning: overflow encountered in exp
  risk_set2 += np.exp(xw[k])
/mnt/CCBdata/projects/conda_envs/BC/lib/python3.11/site-packages/sksurv/linear_model/coxph.py:197: RuntimeWarning: overflow encountered in exp
  risk_set2 += np.exp(xw[k])
/mnt/CCBdata/projects/conda_envs/BC/lib/python3.11/site-packages/sksurv/linear_model/coxph.py:197: RuntimeWarning: overflow encountered in exp
  risk_set2 += np.exp(xw[k])
/mnt/CCBdata/projects/conda_envs/BC/lib/python3.11/site-packages/sksurv/linear_model/coxph.py:197: RuntimeWarning: overflow encountered in exp
  risk_set2 += np.exp(xw[k])
/mnt/CCBdata/projects/conda_envs/BC/lib/python3.11/

coxph: C-index=0.868
rsf: C-index=0.871


XGBoostError: [20:23:41] /workspace/src/c_api/c_api.cu:172: Check failed: p_predt->DeviceCanRead() && !p_predt->HostCanRead(): 
Stack trace:
  [bt] (0) /mnt/CCBdata/projects/conda_envs/BC/lib/python3.11/site-packages/xgboost/lib/libxgboost.so(+0x2bdf8c) [0x7e733a8bdf8c]
  [bt] (1) /mnt/CCBdata/projects/conda_envs/BC/lib/python3.11/site-packages/xgboost/lib/libxgboost.so(+0xade8b8) [0x7e733b0de8b8]
  [bt] (2) /mnt/CCBdata/projects/conda_envs/BC/lib/python3.11/site-packages/xgboost/lib/libxgboost.so(XGBoosterPredictFromCudaArray+0xb2) [0x7e733b0cf472]
  [bt] (3) /mnt/CCBdata/projects/conda_envs/BC/lib/python3.11/lib-dynload/../../libffi.so.8(+0xa052) [0x7e73c66d8052]
  [bt] (4) /mnt/CCBdata/projects/conda_envs/BC/lib/python3.11/lib-dynload/../../libffi.so.8(+0x8925) [0x7e73c66d6925]
  [bt] (5) /mnt/CCBdata/projects/conda_envs/BC/lib/python3.11/lib-dynload/../../libffi.so.8(ffi_call+0xde) [0x7e73c66d706e]
  [bt] (6) /mnt/CCBdata/projects/conda_envs/BC/lib/python3.11/lib-dynload/_ctypes.cp

## Trial

In [2]:
import json
import os
import pandas as pd
import wandb
import joblib
from sklearn.metrics import roc_auc_score, make_scorer
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV, KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sksurv.metrics import concordance_index_censored
from sksurv.util import Surv

from src.architecture.models import make_class_models, make_risk_models
from src.preproc.preproc_utils import FCBFSelector, ToCuPy
from src.utils.utils import setup_hyperparameters
from src.architecture.models import StackingModel

# 0) Setup run hyperparameters and W&B run
config_path = "src/configs/config.yaml"
_, run = setup_hyperparameters(config_path)

# 1) Loading data
df = pd.read_csv("data/data_train.tsv", sep="\t")

# Filtering variables with too many missing values
na_frac = df.drop(columns=["risk_status", "dfs_status", "dfs_time"]).isna().mean()
keep = na_frac[na_frac <= 0.10].index.tolist()
df = df.loc[:, keep + ["risk_status", "dfs_status", "dfs_time"]]

# Removing samples with any missing value
keep = df.drop(columns=["risk_status", "dfs_status", "dfs_time"]).dropna().index.tolist()
df = df.loc[keep, :]

print(f"Training with {df.shape[0]} samples")

y_class = df["risk_status"].copy()
y_event = df["dfs_status"].copy()
y_time = df["dfs_time"].copy()
X = df.drop(columns=["risk_status", "dfs_status", "dfs_time", "os_status"])

# Standard Scale numerical variables
scaler = StandardScaler()
cols = ["edad", "imc", "tamano_tumoral"]
X[cols] = scaler.fit_transform(X[cols])

X_class = X.loc[y_class.notna(), :]
y_class = y_class.loc[y_class.notna()]

X_surv = X.loc[(y_event.notna()) & (y_time.notna()), :]
y_time = y_time.loc[(y_event.notna()) & (y_time.notna())]
y_event = y_event.loc[(y_event.notna()) & (y_time.notna())]
y_surv = Surv.from_arrays(event=y_event.astype(bool), time=y_time)

# 4) Selector for best features
selector = FCBFSelector(mode="rank", threshold=0.0, n_bins=2)

# 5) Models and hyperparameters search
class_models = make_class_models()
risk_models = make_risk_models()

cv_class = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
scorer_class = make_scorer(roc_auc_score)
cv_risk = KFold(n_splits=5, shuffle=True, random_state=0)


def cindex_scorer(estimator, X, y_surv):
    pred = estimator.predict(X)
    return concordance_index_censored(y_surv["event"], y_surv["time"], pred)[0]


scorer_risk = cindex_scorer

# 6) Class models
class_results = {}
class_table_rows = []
# y_class = y_class.astype(int).to_numpy().ravel()

for name, (estimator, search_space) in class_models.items():
    if name == "xgb":
        continue
        pipe = Pipeline(
            [
                ("scaler", StandardScaler()),
                ("selector", selector),
                ("tocupy", ToCuPy()),
                ("model", estimator),
            ]
        )
    else:
        pipe = Pipeline(
            [
                ("scaler", StandardScaler()),
                ("selector", selector),
                ("model", estimator),
            ]
        )

    search = RandomizedSearchCV(
        pipe,
        param_distributions=search_space,
        n_iter=50,
        cv=cv_class,
        scoring="roc_auc",
        n_jobs=-1,
        random_state=0,
        refit=True,
    )

    search.fit(X_class, y_class)
    class_results[name] = {
        "best_score": search.best_score_,
        "best_params": search.best_params_,
        "best_estimator": search.best_estimator_,
    }
    print(f"{name}: AUC={search.best_score_:.3f}")
    if run:
        run.log(
            {
                f"class/{name}/best_auc": search.best_score_,
                f"class/{name}/best_params": json.dumps(search.best_params_, default=str),
            }
        )
        class_table_rows.append([name, search.best_score_, json.dumps(search.best_params_, default=str)])

# 6b) Stacking model con los mejores logreg, svc y rf
required_base = ["logreg", "svc", "rf"]
if all(k in class_results for k in required_base):
    stack_model = StackingModel(
        class_results["logreg"]["best_estimator"],
        class_results["svc"]["best_estimator"],
        class_results["rf"]["best_estimator"],
    )
    stack_model.fit(X_class, y_class)
    stack_auc = roc_auc_score(y_class, stack_model.predict_proba(X_class)[:, 1])
    class_results["stacking"] = {
        "best_score": stack_auc,
        "best_params": {"bases": required_base},
        "best_estimator": stack_model,
    }
    print(f"stacking(logreg,svc,rf): AUC={stack_auc:.3f}")
    if run:
        run.log({"class/stacking/best_auc": stack_auc})
        class_table_rows.append(["stacking(logreg,svc,rf)", stack_auc, json.dumps({"bases": required_base})])

# 7) Risk models
risk_results = {}
risk_table_rows = []

for name, (estimator, search_space) in risk_models.items():

    # Pipeline
    pipe = Pipeline(
        [
            ("scaler", StandardScaler()),
            ("selector", selector),
            ("model", estimator),
        ]
    )

    # Caso 2: con hiperparámetros -> RandomizedSearchCV
    # Caso especial: XGB survival:cox no acepta y_surv (structured)
    search = RandomizedSearchCV(
        pipe,
        param_distributions=search_space,
        n_iter=50,
        cv=cv_risk,
        scoring=scorer_risk,
        n_jobs=-1,
        random_state=0,
        refit=True,
        error_score="raise",
    )
    search.fit(X_surv, y_surv)

    best_score = float(search.best_score_)
    best_params = search.best_params_
    best_estimator = search.best_estimator_

    # Guardar resultados
    risk_results[name] = {
        "best_score": best_score,
        "best_params": best_params,
        "best_estimator": best_estimator,
    }
    print(f"{name}: C-index={best_score:.3f}")

    if run:
        run.log(
            {
                f"risk/{name}/best_cindex": best_score,
                f"risk/{name}/best_params": json.dumps(best_params, default=str),
            }
        )
        risk_table_rows.append([name, best_score, json.dumps(best_params, default=str)])

if run:
    if class_table_rows:
        run.log({"class_results": wandb.Table(columns=["model", "best_auc", "best_params"], data=class_table_rows)})
    if risk_table_rows:
        run.log({"risk_results": wandb.Table(columns=["model", "best_cindex", "best_params"], data=risk_table_rows)})
    run.finish()


Training with 121 samples


/mnt/CCBdata/projects/conda_envs/BC/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


logreg: AUC=0.940
svc: AUC=0.954
rf: AUC=0.925
Training Stacking model...
Finished Stacking model training
stacking(logreg,svc,rf): AUC=0.985


/mnt/CCBdata/projects/conda_envs/BC/lib/python3.11/site-packages/sklearn/model_selection/_search.py:317: UserWarning: The total space of parameters 30 is smaller than n_iter=50. Running 30 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/mnt/CCBdata/projects/conda_envs/BC/lib/python3.11/site-packages/sksurv/linear_model/coxph.py:200: RuntimeWarning: overflow encountered in exp
  risk_set += np.exp(xw[k])
/mnt/CCBdata/projects/conda_envs/BC/lib/python3.11/site-packages/sksurv/linear_model/coxph.py:200: RuntimeWarning: overflow encountered in exp
  risk_set += np.exp(xw[k])
/mnt/CCBdata/projects/conda_envs/BC/lib/python3.11/site-packages/sksurv/linear_model/coxph.py:197: RuntimeWarning: overflow encountered in exp
  risk_set2 += np.exp(xw[k])
/mnt/CCBdata/projects/conda_envs/BC/lib/python3.11/site-packages/sksurv/linear_model/coxph.py:197: RuntimeWarning: overflow encountered in exp
  risk_set2 += np.exp(xw[k])
/mnt/CCBdata/projects/conda_envs/BC/lib/python3.11/si

coxph: C-index=0.872
rsf: C-index=0.866


wandb: ERROR The nbformat package was not found. It is required to save notebook history.


class/logreg/best_auc,▁
class/rf/best_auc,▁
class/stacking/best_auc,▁
class/svc/best_auc,▁
risk/coxph/best_cindex,▁
risk/rsf/best_cindex,▁
class/logreg/best_auc,0.93958
class/logreg/best_params,"{""selector__kbest"": ..."
class/rf/best_auc,0.925
class/rf/best_params,"{""selector__kbest"": ..."
class/stacking/best_auc,0.98529


In [3]:
import joblib

# 8) Persist best estimators for inference
models_dir = "artifacts/models"
os.makedirs(models_dir, exist_ok=True)

for name, result in class_results.items():
    joblib.dump(result["best_estimator"], os.path.join(models_dir, f"class_{name}.pkl"))

for name, result in risk_results.items():
    joblib.dump(result["best_estimator"], os.path.join(models_dir, f"risk_{name}.pkl"))